In [13]:
import torch
import transformers
from transformers import AutoTokenizer
from  langchain import LLMChain, HuggingFacePipeline, PromptTemplate
import pandas as pd

In [14]:
df = pd.read_csv('../../data/df/31_data_df.csv')
df.head()

,Unnamed: 0,Extracted RER #,Links to Document,pdf_link,Tracking ID,Created on,Title,Enforcement Summary,Regulator(s),Total Fine Amount,text_extracted
0,0,RER-1171822,"RER-1171822 Letter of Acceptance, Waiver and ...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1171822,2/10/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",(PRIVATE PLACEMENT OFFERING) From August 2018...,FINRA,"30,000",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...
1,1,RER-1171836,RER-1171836 _ (wolterskluwerfs.com) /,https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1171836,2/10/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",FINRA found the firm failed to reasonably supe...,FINRA,"100,000",TO:\n\nFINANCIAL INDUSTRY REGULATORY AUTHORITY...
2,2,RER-1172079,RER-1172079 Administrative Proceedings In th...,https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1172079,2/11/2023 4:25 AM,Order Instituting Administrative Proceedings I...,"Centaurus Financial, Inc. \n SEC Matter: Betwe...",SEC,"755,509",UNITED STATES OF AMERICA\nBefore the\nSECURITI...
3,3,RER-1173033,"RER-1173033 Letter of Acceptance, Waiver and ...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1173033,2/15/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",(SUITABILITY) From July 2016 through July 2021...,FINRA,"637,356",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...
4,4,RER-1173035,"RER-1173035 Letter of Acceptance, Waiver and...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1173035,2/15/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...","Since June 30, 2020, LIFG has failed to establ...",FINRA,"35,000",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...


In [15]:
# write each summary to the dataframe
gpt_summaries = []
for i in range(len(df['text_extracted'])):
    with open(f'../../samples/Claude/{i}.txt','r') as f:
        gpt_summaries.append(f.read())

df['claude_summary'] = gpt_summaries
df.head()

,Unnamed: 0,Extracted RER #,Links to Document,pdf_link,Tracking ID,Created on,Title,Enforcement Summary,Regulator(s),Total Fine Amount,text_extracted,claude_summary
0,0,RER-1171822,"RER-1171822 Letter of Acceptance, Waiver and ...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1171822,2/10/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",(PRIVATE PLACEMENT OFFERING) From August 2018...,FINRA,"30,000",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...,The Financial Industry Regulatory Authority (F...
1,1,RER-1171836,RER-1171836 _ (wolterskluwerfs.com) /,https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1171836,2/10/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",FINRA found the firm failed to reasonably supe...,FINRA,"100,000",TO:\n\nFINANCIAL INDUSTRY REGULATORY AUTHORITY...,FINRA found that from April 2016 through Decem...
2,2,RER-1172079,RER-1172079 Administrative Proceedings In th...,https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1172079,2/11/2023 4:25 AM,Order Instituting Administrative Proceedings I...,"Centaurus Financial, Inc. \n SEC Matter: Betwe...",SEC,"755,509",UNITED STATES OF AMERICA\nBefore the\nSECURITI...,The SEC found that from June 2016 to July 2019...
3,3,RER-1173033,"RER-1173033 Letter of Acceptance, Waiver and ...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1173033,2/15/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...",(SUITABILITY) From July 2016 through July 2021...,FINRA,"637,356",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...,"FINRA found that from July 2016 to July 2021, ..."
4,4,RER-1173035,"RER-1173035 Letter of Acceptance, Waiver and...",https://rcm.wolterskluwerfs.com/WKFSUploadedFi...,RER-1173035,2/15/2023 4:25 AM,"Letter of Acceptance, Waiver and Consent in th...","Since June 30, 2020, LIFG has failed to establ...",FINRA,"35,000",TO:\n\nRE:\n\nFINANCIAL INDUSTRY REGULATORY AU...,"FINRA found that since June 2020, Long Island ..."


In [16]:
df.to_csv('../../samples/Claude/claude_summaries.csv')

In [3]:
model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)

c:\Users\Yiche\.conda\envs\Butis\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Yiche\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    max_length=4096,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [6]:
template = """
              You are a compliance officer who works at a financial institution. You need to create a summary that covers all important points. Summarize the following text.
              ```{text}```
              SUMMARY:
           """

In [10]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, 
                     llm=llm
                     )

In [11]:
text = df['text_extracted'][0]
len(text)

9289

In [12]:
output = llm_chain.run(text)
print(output)

 The Financial Industry Regulatory Authority (FINRA) has accepted a Letter of Acceptance, Waiver, and Consent from Rosenblatt Securities Inc. (Respondent) regarding alleged rule violations. The Respondent consents to the findings and sanctions without admitting or denying the allegations. The Respondent failed to timely file with FINRA private placement memoranda or any other offering documents for 12 private offerings sold by its registered representatives, and failed to establish and maintain a supervisory system and written procedures reasonably designed to achieve compliance with FINRA Rule 5123. The Respondent agreed to pay a $30,000 fine and to make a corrective action statement.

















































































































































































































































































































